### Imports


In [1]:
import numpy as np
import pandas as pd

import sys
import os




In [ ]:
# Import from src/ and config/
from src.preprocessing import preprocess_aachen_dataset
from config.defaults import Config

# Load the default configuration
config = Config()

# Preprocess the dataset for classification (CNN)
preprocessed_classification = preprocess_aachen_dataset(
    data_path=config.data_path,
    test_cell_count=config.test_cell_count,
    random_state=config.random_state,
    log_transform=config.log_transform,
    classification=True  # Explore classification (CNN) output
)

# Preprocess the dataset for regression (LSTM)
preprocessed_regression = preprocess_aachen_dataset(
    data_path=config.data_path,
    test_cell_count=config.test_cell_count,
    random_state=config.random_state,
    log_transform=config.log_transform,
    classification=False  # Explore regression (LSTM) output
)

## LSTM

### Data Load

In [ ]:
from src.models import load_preprocessed_data

model_type = "regression"

# Load preprocessed data
X_train, X_val, X_test, y_train, y_val, y_test, metadata = load_preprocessed_data(
    model_type, config.eol_capacity
)

2025-03-03 13:03:51,683 - INFO - Loaded preprocessed data and metadata for regression with EOL 0.65


### Grid Search

In [ ]:
from src.grid_search import run_hyperparameter_search

run_hyperparameter_search(config, model_type="lstm")

## CNN

### Data Load

In [2]:
from src.models import load_preprocessed_data
from config.defaults import Config

# Load the default configuration
config = Config()

model_type = "classification"

# Load preprocessed data
X_train, X_val, X_test, y_train, y_val, y_test, metadata = load_preprocessed_data(
    model_type, config.eol_capacity
)

2025-03-05 09:21:56,205 - INFO - Loaded preprocessed data and metadata for classification with EOL 0.65


### Grid Search

In [2]:
from src.grid_search import run_hyperparameter_search

run_hyperparameter_search(config, model_type="cnn")

2025-03-05 09:14:16,925 - INFO - Task type set to: classification, Model type: cnn
2025-03-05 09:14:16,926 - INFO - Loading preprocessed data for hyperparameter tuning...
2025-03-05 09:14:16,928 - INFO - Loaded preprocessed data and metadata for classification with EOL 0.65
2025-03-05 09:14:16,965 - INFO - Starting hyperparameter tuning...
2025-03-05 09:14:16,965 - INFO - Best hyperparameters found: {'conv1_filters': 16, 'conv1_kernel_size': 13, 'l2_reg': 0.0001, 'conv2_filters': 64, 'conv2_kernel_size': 5, 'conv3_filters': 128, 'conv3_kernel_size': 3, 'cnn_dense_units': 128, 'cnn_dropout_rate': 0.3, 'learning_rate': 0.0001}
2025-03-05 09:14:16,966 - INFO - Best hyperparameters saved to: experiments/hyperparameter_tuning/Experiment1_cnn_tuning_eol65_best_params.json


Reloading Tuner from experiments/hyperparameter_tuning/Experiment1_cnn_tuning_eol65/tuner0.json


{'conv1_filters': 16,
 'conv1_kernel_size': 13,
 'l2_reg': 0.0001,
 'conv2_filters': 64,
 'conv2_kernel_size': 5,
 'conv3_filters': 128,
 'conv3_kernel_size': 3,
 'cnn_dense_units': 128,
 'cnn_dropout_rate': 0.3,
 'learning_rate': 0.0001}

### Train model

In [ ]:
from src.models import train_cnn_model

# Update the configuration
config.load_best_params("cnn")

# Input shape for the CNN model
input_shape = (config.seq_len, 1)

cnn_model, history = train_cnn_model(config, X_train, y_train, X_val, y_val, input_shape)

2025-03-05 09:14:17,119 - INFO - CNN model built with tuned config: Config(project_name='Experiment1', data_path='data/raw/Degradation_Prediction_Dataset_ISEA.mat', eol_capacity=0.65, test_cell_count=3, random_state=42, log_transform=False, classification=False, seq_len=120, train_split_ratio=0.8, val_split_ratio=0.2, batch_size=32, max_trials=5, tuning_epochs=10, tuner_directory='experiments/hyperparameter_tuning', conv1_filters=16, conv1_kernel_size=13, conv2_filters=64, conv2_kernel_size=5, conv3_filters=128, conv3_kernel_size=3, l2_reg=0.0001, cnn_dense_units=128, cnn_dropout_rate=0.3)


Loaded best hyperparameters from experiments/hyperparameter_tuning/Experiment1_cnn_tuning_eol65_best_params.json: {'conv1_filters': 16, 'conv1_kernel_size': 13, 'l2_reg': 0.0001, 'conv2_filters': 64, 'conv2_kernel_size': 5, 'conv3_filters': 128, 'conv3_kernel_size': 3, 'cnn_dense_units': 128, 'cnn_dropout_rate': 0.3, 'learning_rate': 0.0001}
Epoch 1/50
131/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3960 - loss: 1.5327
Epoch 1: val_loss improved from inf to 2.04152, saving model to experiments/models/cnn_classification_eol65_20250305_091417_best.keras
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4034 - loss: 1.5092 - val_accuracy: 0.1479 - val_loss: 2.0415
Epoch 2/50
140/142 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5718 - loss: 1.0143
Epoch 2: val_loss did not improve from 2.04152
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5720 - loss: 1.0141 - val_accuracy: 0.1479 - val_loss: 2.0964
Epoch 3/50
134/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.60

2025-03-05 09:14:53,294 - INFO - Final CNN model saved to experiments/models/cnn_classification_eol65_20250305_091453_final.keras


(<Sequential name=sequential, built=True>,
 {'accuracy': [0.48371371626853943,
   0.5825393199920654,
   0.6077997088432312,
   0.6100155115127563,
   0.6439175605773926,
   0.6434744000434875,
   0.6660757660865784,
   0.6773764491081238,
   0.6939951181411743,
   0.71438068151474,
   0.7021936774253845,
   0.7112785577774048,
   0.7261245250701904,
   0.7303345799446106,
   0.7500553727149963,
   0.7387546896934509,
   0.7292266488075256,
   0.7476180195808411,
   0.7573676109313965,
   0.742078423500061,
   0.7624639868736267,
   0.7569244503974915,
   0.7637934684753418,
   0.770440936088562,
   0.7611345052719116,
   0.7766452431678772,
   0.7810769081115723,
   0.7773100137710571,
   0.7806337475776672,
   0.773986279964447,
   0.774429440498352,
   0.7812984585762024,
   0.7835143208503723,
   0.7848438024520874,
   0.7937070727348328,
   0.797252357006073,
   0.7866164445877075,
   0.8052293658256531,
   0.8047861456871033,
   0.8083314895629883,
   0.7930423021316528,
   0.805

In [4]:
from src.models import load_saved_model

cnn_model = load_saved_model("classification", config.eol_capacity, config, model_type="cnn")

2025-03-05 09:23:27,187 - INFO - Loaded saved best CNN model from experiments/models/cnn_classification_eol65_20250305_091417_best.keras


In [7]:
# Evaluate the model
cnn_model.evaluate(X_test, y_test)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6065 - loss: 0.7537 


[0.7765198349952698, 0.6317135691642761]